Step 1: Define the Database Schema

In [21]:
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey, Table
# from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import declarative_base, relationship

Base = declarative_base()



# Association table for many-to-many relationship between books and tags
book_tag_association = Table('book_tag', Base.metadata,
    Column('book_id', Integer, ForeignKey('books.id')),
    Column('tag_id', Integer, ForeignKey('tags.id'))
)

class Author(Base):
    __tablename__ = 'authors'
    id = Column(Integer, primary_key=True, index=True)
    name = Column(String, index=True)
    # Define the relationship from Author to Book
    books = relationship("Book", back_populates="author")

class Book(Base):
    __tablename__ = 'books'
    id = Column(Integer, primary_key=True, index=True)
    title = Column(String, index=True)
    author_id = Column(Integer, ForeignKey('authors.id'))
    # Define the relationship from Book to Author
    author = relationship("Author", back_populates="books")
    tags = relationship('Tag', secondary=book_tag_association, back_populates='books')

class Tag(Base):
    __tablename__ = 'tags'
    id = Column(Integer, primary_key=True, index=True)
    name = Column(String, index=True)
    books = relationship('Book', secondary=book_tag_association, back_populates='tags')


# Create the database
engine = create_engine("sqlite:///books.db")
Base.metadata.create_all(engine)


Step 2: Define Pydantic Models

In [22]:
from pydantic import BaseModel
from typing import List, Optional

class AuthorModel(BaseModel):
    id: Optional[int] = None
    name: str

    class Config:
        from_attributes = True

class TagModel(BaseModel):
    id: Optional[int] = None
    name: str

    class Config:
        from_attributes = True

class BookModel(BaseModel):
    id: Optional[int] = None
    title: str
    author: AuthorModel
    tags: List[TagModel]

    class Config:
        from_attributes = True


Step 3: Interact with the Database

In [23]:
from sqlalchemy.orm import Session

# Start a new session
session = Session(engine)

# Create a new author
new_author = Author(name="John Doe")
session.add(new_author)
session.commit()

# Create new tags
new_tags = [Tag(name="Fiction"), Tag(name="Adventure")]
session.add_all(new_tags)
session.commit()

# Create a new book
new_book = Book(title="The Adventures of John Doe", author=new_author)
new_book.tags.extend(new_tags)
session.add(new_book)
session.commit()

# Retrieve the book from the database
book = session.query(Book).filter(Book.title == "The Adventures of John Doe").first()

# session = Session(engine)
# book = session.query(Book).filter(Book.title == "The Adventures of John Doe").first()

# Convert the SQLAlchemy model to a Pydantic model for serialization
book_model = BookModel.model_validate(book)
print(book_model)




id=1 title='The Adventures of John Doe' author=AuthorModel(id=1, name='John Doe') tags=[TagModel(id=1, name='Fiction'), TagModel(id=2, name='Adventure')]


In [24]:
# Close the session
session.close()